In [ ]:
from PIL import Image
from matplotlib import pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
import numpy as np
import nibabel as nib
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import glob

class BrainTumorDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.subject_dirs = sorted([os.path.join(data_dir, name) for name in os.listdir(data_dir)])
        # print(self.subject_dirs)

    def __len__(self):
        return len(self.subject_dirs)

    def __getitem__(self, idx):
        subject_dir = self.subject_dirs[idx]
        image_path = glob.glob(os.path.join(subject_dir, '*t1n.nii.gz'))[0]
        cropped_image_path = glob.glob(os.path.join(subject_dir, '*t1n-voided.nii.gz'))[0]
        healthy_mask_path = glob.glob(os.path.join(subject_dir, '*healthy.nii.gz'))[0]
        unhealthy_mask_path = glob.glob(os.path.join(subject_dir, '*unhealthy.nii.gz'))[0]


        # 加载图像和掩膜
        image = nib.load(image_path).get_fdata()
        healthy_mask = nib.load(healthy_mask_path).get_fdata()
        unhealthy_mask = nib.load(unhealthy_mask_path).get_fdata()
        cropped_image = nib.load(cropped_image_path).get_fdata()
        mask_affine = nib.load(healthy_mask_path).affine
        image = np.pad(image, ((0, 0), (0, 0), (3, 2)), mode='constant')
        healthy_mask = np.pad(healthy_mask, ((0, 0), (0, 0), (3, 2)), mode='constant')
        cropped_image = np.pad(cropped_image, ((0, 0), (0, 0), (3, 2)), mode='constant')
        unhealthy_mask = np.pad(unhealthy_mask, ((0, 0), (0, 0), (3, 2)), mode='constant')
        # print(mask_affine)


        # 二值化掩膜
        unhealthy_mask = np.where(unhealthy_mask > 0, 1, 0)
        unhealthy_mask_bool = (unhealthy_mask == 1)

        # 使用掩码对图像进行裁剪
        image[unhealthy_mask_bool] = 0

        image = image[np.newaxis, ...]
        cropped_image = cropped_image[np.newaxis, ...]
        healthy_mask = healthy_mask[np.newaxis, ...]

        # image 是健康的图像，即只抠掉肿瘤区域的图像 cropped_image是抠掉要生成区域的图像，mask是健康图像的掩膜
        return image, cropped_image, healthy_mask, mask_affine


In [ ]:
import os
import numpy as np
import nibabel as nib
import glob
# 指定数据路径
data_dir = 'D:\\BraTS\\BraTS-2023_challenge\\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training'
subject_dirs = sorted([os.path.join(data_dir, name) for name in os.listdir(data_dir)])
# 初始化一个大的四维数组来存储所有的patch
all_patches = np.zeros((len(subject_dirs)*16, 120, 120, 40))
temp = 0
# 遍历每个子文件夹
for idx in range(len(subject_dirs)):
    subject_path = os.path.join(data_dir, subject_dir)
    
    # 读取nii.gz文件
    subject_dir = subject_dirs[idx]
    image_path = glob.glob(os.path.join(subject_dir, '*t1n.nii.gz'))[0]
    print(image_path)
    image = nib.load(image_path).get_fdata()
    image = np.pad(image, ((0, 0), (0, 0), (3, 2)), mode='constant')
    # temp = 0
    # 将三维数据分成16个patch
    for i in range(2):
        for j in range(2):
            for k in range(4):
                temp += 1
                patch = image[i*120:(i+1)*120, j*120:(j+1)*120, k*40:(k+1)*40]
                all_patches[temp-1, :, :, :] = patch

# 保存所有的patch到一个大的四维数组中
np.save('all_patches.npy', all_patches)

In [ ]:
import os
import nibabel as nib
import numpy as np
import glob

# 指定数据目录
data_dir = 'D:\\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training'

for subject_dir in os.listdir(data_dir):
    subject_path = os.path.join(data_dir, subject_dir)
    
    # 读取 *mask-healthy.nii.gz 文件
    mask_path = glob.glob(os.path.join(subject_path, '*healthy.nii.gz'))[0]
    mask = nib.load(mask_path).get_fdata()
    
    # 找到mask中非零元素的范围
    nonzero_coords = np.nonzero(mask)
    mask_range_x = np.max(nonzero_coords[0]) - np.min(nonzero_coords[0]) + 1
    mask_range_y = np.max(nonzero_coords[1]) - np.min(nonzero_coords[1]) + 1
    mask_range_z = np.max(nonzero_coords[2]) - np.min(nonzero_coords[2]) + 1
    
    # 检查非零元素的范围是否在 96x96x96 以内
    if mask_range_x > 96 or mask_range_y > 96 or mask_range_z > 96:
        print(f"Subject '{subject_dir}' has a mask with non-zero elements outside the 96x96x96 range: {mask_range_x}x{mask_range_y}x{mask_range_z}")
    # else:
    #     print(f"Subject '{subject_dir}' has a mask with non-zero elements within the 96x96x96 range: {mask_range_x}x{mask_range_y}x{mask_range_z}")

In [2]:
data_dir = r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data'
subject_dirs = sorted([os.path.join(data_dir, name) for name in os.listdir(data_dir)])
# print(subject_dirs)
for idx in range(len(subject_dirs)):
    
    subject_dir = subject_dirs[idx]
    subject_name = os.path.split(subject_dir)[1]
    print(subject_dir)
    image_path = glob.glob(os.path.join(subject_dir, '*t1n.nii.gz'))[0]
    cropped_image_path = glob.glob(os.path.join(subject_dir, '*t1n-voided.nii.gz'))[0]
    healthy_mask_path = glob.glob(os.path.join(subject_dir, '*healthy.nii.gz'))[0]
    unhealthy_mask_path = glob.glob(os.path.join(subject_dir, '*unhealthy.nii.gz'))[0]


    # 加载图像和掩膜
    image = nib.load(image_path).get_fdata().astype(
                    np.float32
                )
    healthy_mask = nib.load(healthy_mask_path).get_fdata().astype(
                    np.float32
                )
    mask_affine = nib.load(healthy_mask_path).affine

    image_mask = np.where(image > 0, 1, 0)
    image_mask_bool = (image_mask != 1) # 没有组织的地方设为0
    healthy_mask[image_mask_bool] = 0.
    
    img = nib.Nifti1Image(healthy_mask, mask_affine)

    nib.save(img, os.path.join(r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain/test_data/test_data/', subject_name, subject_name + '-mask-healthy-modefied.nii.gz'))

    

C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01610-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01657-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01658-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01660-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01661-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01662-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01663-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01664-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01665-000
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01666-000


In [ ]:
def image_preprocess(image):
    t1_clipped = np.clip(
                        image,
                        np.quantile(image, 0.001),
                        np.quantile(image, 0.999),
                    )
    t1_normalized = (t1_clipped - np.min(t1_clipped)) / (
        np.max(t1_clipped) - np.min(t1_clipped)
    )

    return t1_normalized
import os
import nibabel as nib
import numpy as np
import glob
data_dir = r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data'
subject_dirs = sorted([os.path.join(data_dir, name) for name in os.listdir(data_dir)])
for idx in range(len(subject_dirs)):
    subject_dir = subject_dirs[idx]
    # print(subject_dir)
    image_path = glob.glob(os.path.join(subject_dir, '*t1n.nii.gz'))[0]
    cropped_image_path = glob.glob(os.path.join(subject_dir, '*t1n-voided.nii.gz'))[0]
    healthy_mask_path = glob.glob(os.path.join(subject_dir, '*healthy.nii.gz'))[0]
    unhealthy_mask_path = glob.glob(os.path.join(subject_dir, '*unhealthy.nii.gz'))[0]


    # 加载图像和掩膜
    image = nib.load(image_path).get_fdata().astype(
                    np.float32
                )
    healthy_mask = nib.load(healthy_mask_path).get_fdata().astype(
                    np.float32
                )
    unhealthy_mask = nib.load(unhealthy_mask_path).get_fdata().astype(
                    np.float32
                )
    cropped_image = nib.load(cropped_image_path).get_fdata().astype(
                    np.float32
                )
    image = image_preprocess(image)
    cropped_image = image_preprocess(cropped_image)
    mask_affine = nib.load(healthy_mask_path).affine

    nonzero_coords = np.nonzero(healthy_mask)
    center_x = (np.min(nonzero_coords[0]) + np.max(nonzero_coords[0])) // 2
    center_y = (np.min(nonzero_coords[1]) + np.max(nonzero_coords[1])) // 2
    center_z = (np.min(nonzero_coords[2]) + np.max(nonzero_coords[2])) // 2
    image_shape = [240,240,155]
    img_size = 96
    # # 计算裁剪区域的边界
    crop_x1 = max(center_x - int(img_size/2), 0)
    crop_x2 = min(center_x + int(img_size/2), image_shape[0])
    crop_y1 = max(center_y - int(img_size/2), 0)
    crop_y2 = min(center_y + int(img_size/2), image_shape[1])
    crop_z1 = max(center_z - 48, 0)
    crop_z2 = min(center_z + 48, image_shape[2])
    crop_z1 = np.min(nonzero_coords[2])
    crop_z2 = np.max(nonzero_coords[2])

    # 如果裁剪区域小于 96x96x96,则在另一边扩展
    crop_size_x = crop_x2 - crop_x1
    crop_size_y = crop_y2 - crop_y1
    crop_size_z = crop_z2 - crop_z1
    #保存几何坐标信息
    # print(crop_x1, crop_x2, crop_y1, crop_y2, crop_z1, crop_z2)
    geometric_list = [crop_x1, crop_x2, crop_y1, crop_y2, crop_z1, crop_z2]

    if crop_size_x < img_size:
        if center_x - int(img_size/2) < 0:
            crop_x1 = 0
            crop_x2 = img_size
        else:
            crop_x1 = image_shape[0] - int(img_size)
            crop_x2 = image_shape[0]

    if crop_size_y < img_size:
        if center_y - int(img_size/2) < 0:
            crop_y1 = 0
            crop_y2 = img_size
        else:
            crop_y1 = image_shape[1] - int(img_size)
            crop_y2 = image_shape[1]



    image = image[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1:crop_z2]
    healthy_mask = healthy_mask[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1:crop_z2]
    cropped_image = cropped_image[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1:crop_z2]
    unhealthy_mask = unhealthy_mask[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1:crop_z2]


    # 二值化掩膜
    unhealthy_mask = np.where(unhealthy_mask > 0, 1, 0)
    unhealthy_mask_bool = (unhealthy_mask == 1)
    healthy_mask = np.where(healthy_mask > 0, 1, 0)
    healthy_mask_bool = (healthy_mask == 1)

    # 使用掩码对图像进行裁剪
    image[unhealthy_mask_bool] = 0.

    image = image[np.newaxis, ...]
    cropped_image = cropped_image[np.newaxis, ...]
    healthy_mask = healthy_mask[np.newaxis, ...]


    # 保存每个slice的数据
    counter = 0
    for z in range(image.shape[3]):
        slice_healthy_mask = healthy_mask[:, :, :, z]
        slice_image = image[:, :, :, z]
        slice_cropped_image = cropped_image[:, :, :, z]
        if np.sum(slice_healthy_mask) >= 0:
            # 保存为npy文件
            np.savez(os.path.join(subject_dir, str(img_size) + f'_slice_{z}.npz'), 
                     image=slice_image, 
                     healthy_mask=slice_healthy_mask,
                     cropped_image=slice_cropped_image)
        else:
            counter += 1


            # plt.imshow(slice_image[0,:,:], cmap='gray')

In [5]:
import os
import numpy as np
import torch
import glob
import nibabel as nib
import matplotlib.pyplot as plt
def image_preprocess(image):
    t1_clipped = np.clip(
                        image,
                        np.quantile(image, 0.001),
                        np.quantile(image, 0.999),
                    )
    # print(np.max(t1_clipped), np.min(t1_clipped))
    t1_normalized = (t1_clipped - np.min(t1_clipped)) / (
        np.max(t1_clipped) - np.min(t1_clipped)
    )

    return t1_normalized

data_dir = r'C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data'
subject_dirs = sorted([os.path.join(data_dir, name) for name in os.listdir(data_dir)])
# print(subject_dirs)
for idx in range(len(subject_dirs)):
    subject_dir = subject_dirs[idx]
    print(subject_dir)
    image_path = glob.glob(os.path.join(subject_dir, '*t1n.nii.gz'))[0]
    cropped_image_path = glob.glob(os.path.join(subject_dir, '*t1n-voided.nii.gz'))[0]
    healthy_mask_path = glob.glob(os.path.join(subject_dir, '*healthy.nii.gz'))[0]
    
    unhealthy_mask_path = glob.glob(os.path.join(subject_dir, '*unhealthy.nii.gz'))[0]


    # 加载图像和掩膜
    image = nib.load(image_path).get_fdata().astype(
                    np.float32
                )
    healthy_mask = nib.load(healthy_mask_path).get_fdata().astype(
                    np.float32
                )
    unhealthy_mask = nib.load(unhealthy_mask_path).get_fdata().astype(
                    np.float32
                )
    cropped_image = nib.load(cropped_image_path).get_fdata().astype(
                    np.float32
                )
    image = image_preprocess(image)
    cropped_image = image_preprocess(cropped_image)
    mask_affine = nib.load(healthy_mask_path).affine

    nonzero_coords = np.nonzero(healthy_mask)
    center_x = (np.min(nonzero_coords[0]) + np.max(nonzero_coords[0])) // 2
    center_y = (np.min(nonzero_coords[1]) + np.max(nonzero_coords[1])) // 2
    center_z = (np.min(nonzero_coords[2]) + np.max(nonzero_coords[2])) // 2
    image_shape = [240,240,155]
    img_size = 96
    # # 计算裁剪区域的边界
    crop_x1 = max(center_x - int(img_size/2), 0)
    crop_x2 = min(center_x + int(img_size/2), image_shape[0])
    crop_y1 = max(center_y - int(img_size/2), 0)
    crop_y2 = min(center_y + int(img_size/2), image_shape[1])
    crop_z1 = max(center_z - 48, 0)
    crop_z2 = min(center_z + 48, image_shape[2])
    crop_z1 = np.min(nonzero_coords[2])
    crop_z2 = np.max(nonzero_coords[2])

    # 如果裁剪区域小于 96x96x96,则在另一边扩展
    crop_size_x = crop_x2 - crop_x1
    crop_size_y = crop_y2 - crop_y1
    crop_size_z = crop_z2 - crop_z1
    #保存几何坐标信息
    # print(crop_x1, crop_x2, crop_y1, crop_y2, crop_z1, crop_z2)
    geometric_list = [crop_x1, crop_x2, crop_y1, crop_y2, crop_z1, crop_z2]

    if crop_size_x < img_size:
        if center_x - int(img_size/2) < 0:
            crop_x1 = 0
            crop_x2 = img_size
        else:
            crop_x1 = image_shape[0] - int(img_size)
            crop_x2 = image_shape[0]

    if crop_size_y < img_size:
        if center_y - int(img_size/2) < 0:
            crop_y1 = 0
            crop_y2 = img_size
        else:
            crop_y1 = image_shape[1] - int(img_size)
            crop_y2 = image_shape[1]



    image = image[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1:crop_z2]
    healthy_mask = healthy_mask[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1:crop_z2]
    cropped_image = cropped_image[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1:crop_z2]
    unhealthy_mask = unhealthy_mask[crop_x1:crop_x2, crop_y1:crop_y2, crop_z1:crop_z2]
    

    # 二值化掩膜
    unhealthy_mask = np.where(unhealthy_mask > 0, 1, 0)
    unhealthy_mask_bool = (unhealthy_mask == 1)
    healthy_mask = np.where(healthy_mask > 0, 1, 0)
    healthy_mask_bool = (healthy_mask == 1)

    # 使用掩码对图像进行裁剪
    image[unhealthy_mask_bool] = 0.


    # # 对healthy mask 部分加上图像average作为预填充
    # average = np.mean(image)
    # image_preinfilled = image
    # image_preinfilled[healthy_mask_bool] = average

    image = image[np.newaxis, ...]
    cropped_image = cropped_image[np.newaxis, ...]
    healthy_mask = healthy_mask[np.newaxis, ...]


    # 保存每个slice的数据
    counter = 0
    for z in range(image.shape[3]):
        slice_healthy_mask = healthy_mask[:, :, :, z]
        slice_image = image[:, :, :, z]
        slice_cropped_image = cropped_image[:, :, :, z]
        if np.sum(slice_healthy_mask) >= 0:
            # 保存为npy文件
            np.savez(os.path.join(subject_dir, str(img_size) + f'_slice_{z}.npz'), 
                     image=slice_image, 
                     healthy_mask=slice_healthy_mask,
                     cropped_image=slice_cropped_image)
        else:
            counter += 1


            # plt.imshow(slice_image[0,:,:], cmap='gray')
            # plt.show()
    # print(crop_z1, crop_z2)
            

        

C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01610-000
62 88
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01657-000
78 133
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01658-000
59 109
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01659-000
86 117
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01660-000
47 66
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01661-000
66 124
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01662-000
62 90
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01663-000
45 90
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01664-000
49 95
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_brain\test_data\test_data\BraTS-GLI-01665-000
107 125
C:\Users\DELL\Desktop\DDPM\ddpm_brats\DDPM_b

In [ ]:
import os
import numpy as np
import torch

data_dir = 'D:\\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training'
subject_path = 'D:\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training\BraTS-GLI-00003-000\slice_0.npz'
with np.load(subject_path) as data:
    slice_image = data['image']
    slice_healthy_mask = data['healthy_mask']
    slice_cropped_image = data['cropped_image']
    slice_image = torch.from_numpy(slice_image).squeeze(-1)
    # 此处可以对读取的数据进行进一步处理
    print(f"Slice image shape: {slice_image.shape}")
    # print(f"Slice healthy mask shape: {slice_healthy_mask.shape}")
    # print(f"Slice cropped image shape: {slice_cropped_image.shape}")
    # print(f"Slice unhealthy mask shape: {slice_unhealthy_mask.shape}")

In [ ]:
import os
import shutil

# 源文件夹路径
src_dir = 'D:\\ASNR-MICCAI-BraTS2023-Local-Synthesis-Challenge-Training'
# 目标文件夹路径
dest_dir = 'D:\\BraTS\\ASNR-MICCAI-BraTS2023-GLI-Challenge-TrainingData'

# 遍历源文件夹中的所有子文件夹
for subdir in os.listdir(src_dir):
    subdir_path = os.path.join(src_dir, subdir)
    if os.path.isdir(subdir_path):
        # 在目标文件夹中查找同名子文件夹
        dest_subdir_path = os.path.join(dest_dir, subdir)
        if os.path.exists(dest_subdir_path):
            # 复制目标文件夹中的指定文件到源子文件夹
            for file in os.listdir(dest_subdir_path):
                if file.endswith('t1c.nii.gz') or file.endswith('t2f.nii.gz') or file.endswith('t2w.nii.gz'):
                    src_file_path = os.path.join(dest_subdir_path, file)
                    # print(src_file_path)
                    dest_file_path = os.path.join(subdir_path, file)
                    shutil.copy(src_file_path, dest_file_path)
            print(f'Files copied to {subdir_path}')
        else:
            print(f'No matching subdirectory found for {subdir}')